## Подключение всех необходимых библиотек

In [1]:
import librosa
import tqdm
import os

import numpy as np
import pandas as pd

from torch import nn
from torch.functional import F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Hyperparameters

In [2]:
N_MFCCS = 25

# Обработка аудио файлов и создание csv файла (датасета) с признаками

In [3]:
header = 'chroma_stft rms spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(N_MFCCS):
    header += f' mfcc{i}'

## 1. Заполнение csv файла данными

In [17]:
# Функция для извлечения признаков
def extract_features(folder, filename):
    songname = os.path.join(folder, filename)
    y, sr = librosa.load(songname, mono=True)
    rms = librosa.feature.rms(y=y)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=N_MFCCS)

    feature_row_data = [np.mean(chroma_stft), np.mean(rms), np.mean(spec_cent), np.mean(spec_bw), np.mean(rolloff), np.mean(zcr)]
    for e in mfcc:
        feature_row_data.append(np.mean(e))

    feature_row_data.append(folder == 'burr_audio')
    return feature_row_data

In [18]:
count = 0
if not os.path.exists('train_dataset.csv') or \
    pd.read_csv('train_dataset.csv').shape[1] - (6 + 1 + 1) != N_MFCCS: # other features + name + label
    for folder in ['burr_audio', 'normal_audio']:
        columns = (header + " label").split()
        features = pd.DataFrame(columns=columns)
        for filename in tqdm.tqdm(os.listdir(folder), desc=folder):
            try:
                features.loc[filename] = pd.Series(extract_features(folder, filename), index=columns)
            except Exception as k:
                count += 1
        features.to_csv('train_dataset.csv')

burr_audio:   0%|          | 33/7818 [00:00<00:47, 163.12it/s]/Users/jafarkor/GitHub/ai-challenge-anticartavost-/venv/lib/python3.11/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/jafarkor/GitHub/ai-challenge-anticartavost-/venv/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
burr_audio:   1%|          | 91/7818 [00:00<00:42, 182.74it/s]/Users/jafarkor/GitHub/ai-challenge-anticartavost-/venv/lib/python3.11/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/Users/jafarkor/GitHub/ai-challenge-anticartavost-/venv/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
burr_audio:   6%|▌         | 436/7818 [00:02<00:39, 186.67it/s]/Users/jafarkor/G

In [19]:
print(count)

53


## Обработка датасета и разделение на тренировочную и тестовую выборки

In [28]:
df = pd.read_csv("train_dataset.csv", index_col=0)
X = df.drop(columns=["label"])
y = df["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Масштабирование признаков

In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [438]:
"""pca = PCA(n_components=9)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)"""

'pca = PCA(n_components=9)\nX_train = pca.fit_transform(X_train)\nX_test = pca.transform(X_test)'

In [31]:
import torch.utils
import torch.utils.data


class MyDataset(torch.utils.data.Dataset):

    def __init__(self, features, labels, device=torch.device("cpu")):
        self.features = torch.tensor(features, dtype = torch.float32).to(device)
        self.labels = torch.tensor(labels, dtype = torch.float32).to(device)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, ind):
        return self.features[ind], self.labels[ind]

In [33]:
import torch.utils
import torch.utils.data
import torch.nn.functional

device = torch.device("mps")
train_dataset = MyDataset(X_train.values, y_train, device)
test_dataset = MyDataset(X_test.values, y_test, device)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)

/var/folders/2z/1mq4rcvj0f75_rlwvf4546gc0000gn/T/ipykernel_32673/3082938681.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.labels = torch.tensor(labels, dtype = torch.float32).to(device)


## Архитектура нашей нейронной сети и ее обучение

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim

class RecognizeNet(nn.Module):
    def __init__(self):
        super(RecognizeNet, self).__init__()
        self.layer1 = nn.Linear(6 + N_MFCCS, 100)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)  # Dropout для первого слоя

        self.layer2 = nn.Linear(100, 200)
        self.act2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)  # Dropout для второго слоя

        self.layer3 = nn.Linear(200, 100)
        self.act3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.5)  # Dropout для третьего слоя

        self.layer4 = nn.Linear(100, 50)
        self.act4 = nn.ReLU()
        self.dropout4 = nn.Dropout(0.5)  # Dropout для четвертого слоя

        self.layer5 = nn.Linear(50, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer1(x)
        x = self.act1(x)

        x = self.layer2(x)
        x = self.act2(x)
        x = self.dropout2(x)

        x = self.layer3(x)
        x = self.act3(x)

        x = self.layer4(x)
        x = self.act4(x)
        x = self.dropout4(x)

        x = self.layer5(x)
        x = self.sigmoid(x)
        return x

In [35]:
model = RecognizeNet()
lr = 0.01
model.to(device)
criterion = nn.BCELoss()  # Предполагаем, что решаем задачу бинарной классификации
optimizer = optim.Adam(model.parameters(), lr=lr)
total_epochs = 0

In [36]:
total_step = len(train_loader)
epochs = 500
loss_fn = nn.BCELoss()
acc_list = []
pbar = tqdm.tqdm(range(epochs))
for epoch in pbar:
    for i, batch in enumerate(train_loader):

        x, y = batch
        preds = model(x).squeeze()
        loss = loss_fn(preds, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total = y.size(0)
        predicted = (torch.flatten((preds.data > 0.5)))
        true = y
        correct = (predicted == true).sum().item()
        acc_list.append(correct / total)
    total_epochs += 1
    if (epoch + 1) % 10 == 0:
        pbar.set_description('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(total_epochs, (total_epochs // epochs + 1) * epochs, loss.item(),
                        (correct / total) * 100))

Epoch [80/500], Loss: 0.0000, Accuracy: 100.00%:  18%|█▊        | 88/500 [00:34<02:39,  2.58it/s]


KeyboardInterrupt: 

## Тестирование нашей нейронки

In [37]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for x, y in test_loader:
        y_pred = model(x)
        predicted = (torch.flatten((F.sigmoid(y_pred.data) > 0.5)))
        total += y.size(0)
        correct += (predicted == y).sum().item()

    print('Test Accuracy of the model on the test data: {} %'.format((correct / total) * 100))

Test Accuracy of the model on the test data: 100.0 %


# Submission

In [8]:
data_scoring = pd.read_csv("test_dataset.csv", index_col=0)
sample_submission = pd.read_csv("test.csv", header=None)
sample_submission.columns = ['Filename', 'Label']


model.eval()
with torch.no_grad():
    results = model(torch.FloatTensor(data_scoring.values).to(device)).cpu().squeeze()


submission = pd.DataFrame({"Filename": data_scoring.index, "Label": pd.Series(results).astype(int)})
submission = submission.sort_values(by="Filename")
#ВСЕ ОК)))

sample_submission = sample_submission.sort_values(by="Filename")
sample_submission["Label"] = submission["Label"]
sample_submission = sample_submission.sort_index()


sample_submission.to_csv("submission.csv", index=False, header=False)

NameError: name 'model' is not defined